<a href="https://colab.research.google.com/github/prakashaditya369/AdditionQuiz/blob/master/BCS_steinmetz_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steinmetz Project Assignment 1
## LIF Neuron

Click on **Copy to Drive** to make a copy and modify the code in that.  
**Acknowledgement**: Based on Neuromatch Tutorials.  
Happy Learning. :)

---
## Assignment Objective
The objective of the assignment is to make you familiar with Python while implementing a **Leaky Integrate and Fire** neuron.

---
## Imports and helper functions
Please execute the cell(s) below to initialize the notebook environment.

In [ ]:
# @title Import Libraries and Figure Settings
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

---
## Neuron model
A *membrane equation* and a *reset condition* define our *leaky-integrate-and-fire (LIF)* neuron:


\begin{align*}
\\
&\tau_m\,\frac{d}{dt}\,V(t) = E_{L} - V(t) + R\,I(t) &\text{if }\quad V(t) \leq V_{th}\\
\\
&V(t) = V_{reset} &\text{otherwise}\\
\\
\end{align*}

where $V(t)$ is the membrane potential, $\tau_m$ is the membrane time constant, $E_{L}$ is the leak potential, $R$ is the membrane resistance, $I(t)$ is the synaptic input current, $V_{th}$ is the firing threshold, and $V_{reset}$ is the reset voltage. We can also write $V_m$ for membrane potential - very convenient for plot labels.

The membrane equation is an *ordinary differential equation (ODE)* that describes the time evolution of membrane potential $V(t)$ in response to synaptic input and leaking of change across the cell membrane.

**Note that, in this assignment, we will be working with a single neuron.**

### Exercise 1
We start by defining and initializing the main simulation variables.

**Suggestions**
* Modify the code below to print the simulation parameters

In [ ]:
# t_max = 150e-3   # second
# dt = 1e-3        # second
# tau = 20e-3      # second
# el = -60e-3      # milivolt
# vr = -70e-3      # milivolt
# vth = -50e-3     # milivolt
# r = 100e6        # ohm
# i_mean = 25e-11  # ampere

# print(t_max, dt, tau, el, vr, vth, r, i_mean)

**SAMPLE OUTPUT**

```
0.15 0.001 0.02 -0.06 -0.07 -0.05 100000000.0 2.5e-10
```

### Exercise 2
![synaptic input](https://github.com/mpbrigham/colaboratory-figures/raw/master/nma/python-for-nma/synaptic_input.png)

We start with a sinusoidal model to simulate the synaptic input $I(t)$ given by:
\begin{align*}
\\
I(t)=I_{mean}\left(1+\sin\left(\frac{2 \pi}{0.01}\,t\right)\right)\\
\\
\end{align*}

Compute the values of synaptic input $I(t)$ between $t=0$ and $t=0.009$ with step $\Delta t=0.001$.

**Suggestions**
* Loop variable `step` for 25 steps (`step` takes values from `0` to `24`)
* At each time step
    * Compute the value of `t` with variables `step` and `dt`
    * Compute the value of `i` using the formula given above.
    * Use `np.pi` and `np.sin` for evaluating $\pi$ and $\sin(\cdot)$, respectively.
    * `i` vs `t` graph is plotted.

In [ ]:
# initialize t
t = 0

####### Plotting Details ########
plt.figure()
plt.title('Sinusoidal I(t)')
plt.xlabel('time (s)')
plt.ylabel('$I$ (A)');
##################################

# loop for 10 steps, variable 'step' takes values from 0 to 9
for step in range(25):
  # t = ...            # Current time would be step * dt.
  # i = ...            # Find i using the formula
  plt.plot(t, i,'ko')
plt.show()

**SAMPLE OUTPUT**

<img alt='Solution hint' align='left' width=559 height=416 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W0D1_PythonWorkshop1/static/W0D1_Tutorial1_Solution_23446a7e_0.png>

### Exercise 3
Print formatting is handy for displaying simulation parameters in a clean and organized form. Python 3.6 introduced the new string formatting [f-strings](https://www.python.org/dev/peps/pep-0498). Since we are dealing with type `float` variables, we use `f'{x:.3f}'` for formatting `x` to three decimal points, and `f'{x:.4e}'` for four decimal points but in exponential notation.
```
x = 3.14159265e-1
print(f'{x:.3f}')
--> 0.314

print(f'{x:.4e}')
--> 3.1416e-01
```

Repeat the loop from the previous exercise and print the `t` values with three decimal points, and synaptic input $I(t)$ with four decimal points in exponential notation.

For additional formatting options with f-strings see [here](http://zetcode.com/python/fstring/).

**Suggestions**
* Print `t` and `i` with help of *f-strings* formatting

In [ ]:
# initialize step_end
step_end = 10

# loop for step_end steps
for step in range(step_end):
  t = ...  # Same as done above
  i = ...  # Same as done above
  print(...)

**SAMPLE OUTPUT**

```
0.000 2.5000e-10
0.001 3.9695e-10
0.002 4.8776e-10
0.003 4.8776e-10
0.004 3.9695e-10
0.005 2.5000e-10
0.006 1.0305e-10
0.007 1.2236e-11
0.008 1.2236e-11
0.009 1.0305e-10
```

## ODE integration without spikes
In the next exercises, we now simulate the evolution of the membrane equation in discrete time steps, with a sufficiently small $\Delta t$.

We start by writing the time derivative $d/dt\,V(t)$ in the membrane equation without taking the limit $\Delta t \to 0$:

\begin{align*}
\\
\tau_m\,\frac{V\left(t+\Delta t\right)-V\left(t\right)}{\Delta t} &= E_{L} - V(t) + R\,I(t) \qquad\qquad (1)\\
\\
\end{align*}

The value of membrane potential $V\left(t+\Delta t\right)$ can be expressed in terms of its previous value $V(t)$ by simple algebraic manipulation. For *small enough* values of $\Delta t$, this provides a good approximation of the continuous-time integration.

This operation is an integration since we obtain a sequence $\{V(t), V(t+\Delta t), V(t+2\Delta t),...\}$ starting from the ODE. Notice how the ODE describes the evolution of $\frac{d}{dt}\,V(t)$, the derivative of $V(t)$, but not directly the evolution of $V(t)$. For the evolution of $V(t)$ we need to integrate the ODE, and in this tutorial, we will do a discrete-time integration using the Euler method. See [Numerical methods for ordinary differential equations](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations) for additional details.

### Exercise 4
Compute the values of $V(t)$ between $t=0$ and $t=0.01$ with step $\Delta t=0.001$ and $V(0)=E_L$.

We will write a `for` loop from scratch in this exercise. The following three formulations are all equivalent and loop for three steps:
```
for step in [0, 1, 2]:
  print(step)

for step in range(3):
  print(step)

start = 0
end = 3
stepsize = 1

for step in range(start, end, stepsize):
  print(step)
```


**Suggestions**
* Reorganize the Eq. (1) to isolate $V\left(t+\Delta t\right)$ on the left side, and express it as function of $V(t)$ and the other terms
\begin{align*}
\\
V\left(t+\Delta t\right) &= {V\left(t\right)} + \frac{{\Delta t}}{\tau_m} *(E_{L} - V(t) + R\,I(t))  \qquad\qquad (2)\\
\\
\end{align*}
* Compute the required number of steps with`int(t_max/dt)`
* Initialize the membrane potential variable `v` to leak potential `el`
* At each time step
    * Compute the current value of `t`, `i`
    * Plot the value of `v` vs `t`.
    * Update the value of `v`

In [ ]:
# initialize step_end and v
step_end = ...   #Compute the number of steps
v = el

# initialize the figure
plt.figure()
plt.title('$V_m$ with sinusoidal I(t)')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)');

# loop for step_end steps
for step in range(step_end):
  t = ...    # Same as above
  i = ...    # Same as above
  # Complete this line and uncomment
  plt.plot(t,v,'k.')

  v = ...  #Find v using the formula given above.

t = t + dt
# Complete this line and uncomment
plt.plot(t,v,'k.')
plt.show()

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W0D1_PythonWorkshop1/static/W0D1_Tutorial1_Solution_1046fd94_0.png>

---
## Random synaptic input
From the perspective of neurons, synaptic input is random (or stochastic). We'll improve the synaptic input model by introducing random input current with statistical properties similar to the previous exercise:

\begin{align*}
\\
I(t)=I_{mean}\left(1+0.1\sqrt{\frac{t_{max}}{\Delta t}}\,\xi(t)\right)\qquad\text{with }\xi(t)\sim U(-1,1)\\
\\
\end{align*}

where $U(-1,1)$ is the [uniform distribution](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)) with support $x\in[-1,1]$.

Random synaptic input $I(t)$ results in random time course for $V(t)$.

### Exercise 5
Plot the values of $V(t)$ between $t=0$ and $t=t_{max}-\Delta t$ with random input $I(t)$.

Initialize the (pseudo) random number generator (RNG) to a fixed value to obtain the same random input each time.

The function `np.random.seed()` initializes the RNG, and `np.random.random()` generates samples from the uniform distribution between `0` and `1`.

**Suggestions**
* Use `np.random.seed()` to initialize the RNG to `0`
* Use `np.random.random()` to generate random input in range `[0,1]` at each timestep
* Multiply random input by an appropriate factor to expand the range to `[-1,1]`
  *  To generate random number between `[L, U]`, use `(U-L)*random_number + L`, where random_number is between `[0,1]`.
* Verify that $V(t)$ has a random time course by changing the initial RNG value
* Alternatively, comment RNG initialization by typing `CTRL` + `\` in the relevant line

In [ ]:
# set random number generator
np.random.seed(2020)

# initialize step_end and v
step_end = int(t_max / dt)
v = el

# initialize the figure
plt.figure()
plt.title('$V_m$ with random I(t)')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)')

# loop for step_end steps
for step in range(step_end):
  t = step * dt
  # Complete this line and uncomment
  plt.plot(t, v, 'k.')

  i = ... # Find i using above formula
  v = ... # As calculated in previous exercise.

plt.show()

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src='https://drive.google.com/uc?id=1hnB78v251QOx_65BX79XSJbSZ61tLlgu'>

### Exercise 6
### Using List
After each iteration append the value of `t`, `i`, `v` in lists: `time_range`, `i_n`, `v_n` respectively.

**Suggestions**
* Find `i` and `v` same as above.
* Use `list_name.append(element)` to append elements to list.

In [ ]:
# set random number generator
np.random.seed(2025)

# initialize step_end and v
step_end = int(t_max / dt)
v = el
time_range = []
i_n = []
v_n = []

# loop for step_end steps
for step in range(step_end):
  t = step * dt

  i = ... # Same as above
  v = ... # Same as above
  ... # Append t
  ... # Append i
  ... # Append v


fig_w, fig_h = plt.rcParams['figure.figsize']
plt.figure(figsize=(fig_w, 2 * fig_h))

plt.subplot(2,1,1)
plt.plot(time_range, i_n)
plt.title('Random $I(t)$')
plt.xlabel('time (s)')
plt.ylabel('$I$ (A)')

plt.subplot(2,1,2)
plt.plot(time_range, v_n)
plt.title('$V_m$ with random I(t)')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)')

plt.show()

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src='https://drive.google.com/uc?id=1IFQHK5uAW_nLQfqxHMbmE5hy2PXWJVkX'>

---
## Introduce spikes
A spike takes place whenever $V(t)$ crosses $V_{th}$. In that case, a spike is recorded and $V(t)$ resets to $V_{reset}$ value. This is summarized in the *reset condition*:
$$V(t) = V_{reset}\quad \text{ if } V(t)\geq V_{th}$$

For more information about spikes or action potentials see [here](https://en.wikipedia.org/wiki/Action_potential) and [here](https://www.khanacademy.org/test-prep/mcat/organ-systems/neuron-membrane-potentials/a/neuron-action-potentials-the-creation-of-a-brain-signal).


![spikes cartoon](https://github.com/mpbrigham/colaboratory-figures/raw/master/nma/python-for-nma/spikes_carton.png)

### Exercise 7
Implement spikes for a single neuron.

**Suggestions**
* At each time step loop all neurons to:
  * Reset $V_n(t)$ to $V_{reset}$ if $V_n(t)\geq V_{th}$
  * Add spike time to `spikes`.

In [ ]:
# set random number generator
np.random.seed(2020)

# initialize step_end and v
step_end = int(t_max / dt)
v = el
time_range = []
v_n = []
spikes = []
# loop for step_end steps
for step in range(step_end):
  t = step * dt
  # Complete this line and uncomment

  i = i_mean * (1 + 0.1 * (t_max / dt)**(0.5) * (2 * np.random.random() - 1))
  v = v + (dt / tau) * (el - v + r * i)
  if (...):      # Reset Condition
    ...     # Reset potential
    ...     #Append current time into 'spikes'
  time_range.append(t)
  v_n.append(v)


plt.subplot(2,1,1)
plt.xlim((0, t_max))
plt.plot(time_range, v_n)
plt.title('$V(t)$')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)')

plt.subplot(2,1,2)
plt.xlim((0, t_max))
plt.eventplot(spikes)
plt.show()

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src='https://drive.google.com/uc?id=1IbA4BOwdlptPK_vfvr1ckbdUoUfCL1Do'>

---
## Using Functions

### Exercise 8
Re-organize part of the code from the previous exercise with functions.

**Suggestions**
*  Complete the functions `get_i()` and `ode_and_reset_step` using ideas from above exercises.
*   Call the function in the loop to get value of `i` and `v`.
*   Using ideas from above plotting examples, complete the `plt.plot()` calls.

In [ ]:
def get_i(i_mean, t_max, dt):
  """
  Return a random synpatic input

  Args:
    i_mean (float)
      mean synaptic input

    t_max (float)
      maximum time limit

    dt (float)
      time step increment

  Returns:
    i (float)
      random synaptic input
  """
  i = ... # complete the function here.
  return i

def ode_and_reset_step(v, i, dt, t, spikes, tau, el, r):
  """
  Evolves membrane potential by one step of discrete time integration and resets if potential crosses threshold.
  In case of spikes, add the time to spikes list.

  Args:
    v (numpy array of floats)
      membrane potential at previous time step of shape (neurons)

    v (numpy array of floats)
      synaptic input at current time step of shape (neurons)

    dt (float)
      time step increment
    
    spikes (list of floats)
       list of times when spikes is created.

  Returns:
    v (float)
      membrane potential at current time step of shape (neurons)
    spikes(list of floats)
      list of times when spikes is created.
  """
  '''One step ode, then check for threshold potential and reset if reuired. 
  Then append current time to spikes.'''
  ...
  return v, spikes

# set random number generator
np.random.seed(2030)

# initialize step_end and v
step_end = int(t_max / dt)
v = el
time_range = []
v_n = []
spikes = []
# loop for step_end steps
for step in range(1,step_end):
  t = step * dt
  # Complete this line and uncomment

  i = ...  #Call the function
  v, spikes = ...  #Call the function
  time_range.append(t)
  v_n.append(v)


plt.subplot(2,1,1)
plt.xlim((0, t_max))
plt.plot(...)       # write the arguments properly
plt.title('$V(t)$')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)')

plt.subplot(2,1,2)
plt.xlim((0, t_max))
plt.eventplot(...)     # write the arguments properly
plt.show()


*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src='https://drive.google.com/uc?id=1JhPltu83rI0I2cyruMiW2y9nb7R90U5-'>

---
## Using classes
Using classes helps with code reuse and reliability. Well-designed classes are like black boxes in that they receive inputs and provide expected outputs. The details of how the class processes inputs and produces outputs are unimportant.

See additional details here: [A Beginner's Python Tutorial/Classes](https://en.wikibooks.org/wiki/A_Beginner%27s_Python_Tutorial/Classes)

*Attributes* are variables internal to the class, and *methods* are functions internal to the class.

### Exercise 9
In this exercise we'll practice with Python classes by implementing `LIFNeurons`, a class that evolves and keeps state of multiple realizations of LIF neurons.

Class Variables:
```
self.v             current membrane potential
self.t             running time of the simulation
self.steps         simulation step
```
Class method:
```
self.ode_step()    performs single step discrete time integration
                   for provided dt
```

**Suggestions**
* Initialise the class variables based on given examples.
*  Complete `ode_step()` with the help of above exercises.
*  Initiaise a class object.For a class with name `Neuron`, object is defined using `neuron = Neuron(arguments)`. 
*  Call the class method (`neuron.method_name(arguments)`) and then use class varibales (`neuron.variable_name`) for plotting the graphs.

In [ ]:
# simulation class
class LIFNeuron:
  """
  Keeps track of membrane potential for a single LIF neuron,
  and performs single step discrete time integration.
  """
  def __init__(self, tau=20e-3, el=-60e-3, vr=-70e-3, vth=-50e-3, r=100e6, i_mean = 25e-11):

    # neuron parameters
    self.tau = tau        # second
    # Initialise the parameters.
    self.el = ...          # millivolt
    self.vr = ...         # millivolt
    self.vth = ...        # millivolt
    self.r = ...            # ohm
    self.i_mean = ...  #Ampere

    # state variables
    self.v = el
    self.steps = 0
    self.t = 0
    self.i = 0
    self.i_n = []
    self.time_range = []
    self.v_n = []
    self.spikes = []

  def ode_step(self, dt, t_max):

    # update running time and steps
    self.i = ... # Find value of i.
    # one step of discrete time integration of dt
    self.v = ... # Find value of v.
    if self.v >= self.vth:
      self.v = self.vr
      self.spikes.append(self.t)
    self.time_range.append(self.t)
    self.i_n.append(self.i)
    self.v_n.append(self.v)
    self.t += dt
    self.steps += 1

# set random number generator
np.random.seed(2040)


neuron = ...  # initialize neuron
step_end = int(t_max / dt)

# loop time steps
for step in range(step_end):
  # Complete the method and uncomment
  ...   # Call ode_step() with proper arguments

#plottings
fig_w, fig_h = plt.rcParams['figure.figsize']
plt.figure(figsize=(fig_w, 3 * fig_h))

plt.subplot(3,1,1)
plt.xlim((0, t_max))
plt.plot(...)               # write the arguments properly
plt.title('Random $I(t)$')
plt.xlabel('time (s)')
plt.ylabel('$I$ (A)')

plt.subplot(3,1,2)
plt.plot(...)              # write the arguments properly
plt.title('$V(t)$')
plt.xlabel('time (s)')
plt.ylabel('$V_m$ (V)')

plt.subplot(3,1,3)
plt.xlim((0, t_max))
plt.eventplot(...)          # write the arguments properly
plt.show()

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src='https://drive.google.com/uc?id=1sWZxtOu5uDNGQKEpBR96Yw691CH0_Z2g'>

Keep Learning :)